In [8]:

import pandas as pd
import os
import json

''' Notes
unit of weight material density for che is kg/m3
unit of low gravity solid percent is percent, and only use the first float in the front of '/'
unit of salinity is a percent and only use the first float in the front of '/'
'''

def convert_lbbbl_2_kgm3(lbbbl: float):
    return lbbbl * 2.85

# for alaska wells
def get_base_fluid_type(mud_class):
    if mud_class == 'waterbased':
        return 'Water-Based Mud'
    elif mud_class == 'oilbased':
        return 'Escaid 110'
    else:
        raise Exception()

def merge_fluid_properties(
        report_path,
        solid_percent,
        oil_water_ratio,
        salt_type,
        salinity,
        low_gravity_density,
        weight_material_density
          ):
    with open(report_path, 'r') as f:
        report = json.load(f)

    assert len(report['fluid']) == 1

    fluid_witsml_obj = report['fluid'][0]

    assert 'tempVis' in fluid_witsml_obj.keys()
    assert 'WaterPhaseSalinity' not in fluid_witsml_obj.keys()
    assert 'SolidsLowGravPc' not in fluid_witsml_obj.keys()

    for i, extension_name_value in enumerate(fluid_witsml_obj['extensionNameValue']):
        assert extension_name_value['name'] != 'MudInTemperature'
        assert extension_name_value['name'] != 'BaseFluid'
        if extension_name_value['name'] == 'SaltType':
            fluid_witsml_obj['extensionNameValue'].remove(extension_name_value)
        assert extension_name_value['name'] != 'WeightMaterialDensity'


    oil_water_ratio_strs = oil_water_ratio.split('/')
    oil_pc = float(oil_water_ratio_strs[0]) / 100
    water_pc = float(oil_water_ratio_strs[1]) / 100

    fluid_witsml_obj['oilPc'] = {
                'value': oil_pc
            }
    fluid_witsml_obj['waterPc'] = {
                'value': water_pc 
            }
    fluid_witsml_obj['waterPhaseSalinity'] = {
        'value': float(salinity.split('/')[0]) / 100
    }
    fluid_witsml_obj['solidsLowGravPc'] = {
        'value': float(low_gravity_density.split('/')[0]) / 100
    }

    if '/' in str(weight_material_density):
        weight_material = weight_material_density.split('/')
        weight_material = float(weight_material[1]) if len(weight_material)>1 else 0
    else:
        weight_material = float(weight_material_density)

    fluid_witsml_obj['extensionNameValue'] +=[
        {
            'name': 'MudInTemperature',
            'value': fluid_witsml_obj['tempVis']
        },
        {
            'name': 'BaseFluid',
            'value': {
                'value': get_base_fluid_type(fluid_witsml_obj['mudClass'])
            }
        },
        {
            "name": "SaltType",
            "value": {
                "value": row['Salt type']
            },
        },
        {
            'name': 'WeightMaterialDensity',
            'value': {
                'value': convert_lbbbl_2_kgm3(weight_material)
            }
        }
    ]

    return report

well_data_dir = r'C:\NotOneDrive\Data\1c-157_Run5_bha'
more_fluid_properties_path = r"C:\NotOneDrive\Data\1c-157_Run5_bha\READY_1C-157  Run 5_FluidsMoreProperties.xlsx"

more_fluid_properties = pd.read_excel(more_fluid_properties_path, )
more_fluid_properties.fillna("",inplace=True)
# more_fluid_properties = more_fluid_properties.dropna()

for index, row in more_fluid_properties.iterrows():
    if index == 0: continue
    report_path = os.path.join(well_data_dir, row['ReportName'])
    save_name = rf'more_' + row['ReportName']
    save_path = os.path.join(well_data_dir, save_name)
    with open(report_path, 'r') as f:
        report = json.load(f)

    assert len(report['fluid']) == 1

    fluid_witsml_obj = report['fluid'][0]

    assert 'tempVis' in fluid_witsml_obj.keys()
    assert 'WaterPhaseSalinity' not in fluid_witsml_obj.keys()
    assert 'SolidsLowGravPc' not in fluid_witsml_obj.keys()

    for i, extension_name_value in enumerate(fluid_witsml_obj['extensionNameValue']):
        assert extension_name_value['name'] != 'MudInTemperature'
        assert extension_name_value['name'] != 'BaseFluid'
        if extension_name_value['name'] == 'SaltType':
            fluid_witsml_obj['extensionNameValue'].remove(extension_name_value)
        assert extension_name_value['name'] != 'WeightMaterialDensity'


    oil_water_ratio_strs = row['Oil water ratio'].split('/')
    oil_pc = float(oil_water_ratio_strs[0]) / 100
    water_pc = float(oil_water_ratio_strs[1]) / 100

    fluid_witsml_obj['oilPc'] = {
                'value': oil_pc
            }
    fluid_witsml_obj['waterPc'] = {
                'value': water_pc 
            }
    fluid_witsml_obj['waterPhaseSalinity'] = {
        'value': float(row['Salinity'].split('/')[0]) / 100
    }
    fluid_witsml_obj['solidsLowGravPc'] = {
        'value': float(row['LGS'].split('/')[0]) / 100
    }

    if '/' in str(row['Weight material']):
        weight_material = row['Weight material'].split('/')
        weight_material = float(weight_material[1]) if len(weight_material)>1 else 0
    else:
        weight_material = float(row['Weight material'])

    fluid_witsml_obj['extensionNameValue'] +=[
        {
            'name': 'MudInTemperature',
            'value': fluid_witsml_obj['tempVis']
        },
        {
            'name': 'BaseFluid',
            'value': {
                'value': get_base_fluid_type(fluid_witsml_obj['mudClass'])
            }
        },
        {
            "name": "SaltType",
            "value": {
                "value": row['Salt type']
            },
        },
        {
            'name': 'WeightMaterialDensity',
            'value': {
                'value': convert_lbbbl_2_kgm3(weight_material)
            }
        }
    ]
    with open(save_path, 'w') as f:
        json.dump(report, f, indent=4)